# 1. Causal inference: Data Simulation and Average Treatment Effects

Very useful book: https://users.aalto.fi/~ave/ROS.pdf. Lectures and seminars are based on the chapters 18-21.

## Simulating Data

Plan is to look at how we can create:
+ samples from normal distributions
+ several variables from normal distributions with specified correlation
+ dataset with 2 correlated variables with specified treatment effect
+ vector z for random treatment assignment with specified probability to being assigned to the treatment
+ imitate population with different stratas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Sample from normal distribution

In [ ]:
np.random.seed(10)
# here - mean=0, sd=1, size of the sample=10
x = np.random.normal(loc=0.0, scale=1.0, size=10)

In [ ]:
plt.hist(x)

In [ ]:
x = np.random.normal(loc=0.0, scale=1.0, size=1000)
plt.hist(x)

**Creating several variables with specified correlation**


Pearson's correlation: 

$$\rho(X, Y) = \frac{\text{cov}(X, Y)}{\sigma_x \sigma_y}$$

so, $$\text{cov}(X, Y) = \rho(X, Y) \sigma_x \sigma_y$$

For creating multivariate Normal distribution $X \sim N(\mu, \Sigma)$ we need the vector of means $\mu$ (mean value for each variable) and covariance matrix $\Sigma$. For example, for 3-dimensional Multivariate Normal distribution covariance matrix would look like:


$$
\begin{pmatrix}
\text{cov}(x_1, x_1) & \text{cov}(x_1, x_2) & \text{cov}(x_1, x_3)\\
\text{cov}(x_2, x_1) & \text{cov}(x_2, x_2) & \text{cov}(x_2, x_3)\\
\text{cov}(x_3, x_1) & \text{cov}(x_3, x_2) & \text{cov}(x_3, x_3)
\end{pmatrix}
$$


So, if we want to create, for example, 3 variables $X_1, X_2, Y_2$ with pair-wise correlations $\rho(X_1, X_2) = a$, $\rho(X_1, X_3) = b$, $\rho(X_2, X_3) = c$, then we need to create the following covariance matrix:


$$
\begin{pmatrix}
\sigma_{x1} \sigma_{x1} & a \sigma_{x1} \sigma_{x2} & b \sigma_{x1} \sigma_{x3}\\
a \sigma_{x1} \sigma_{x2} & \sigma_{x2} \sigma_{x2} & c \sigma_{x2} \sigma_{x3})\\
b \sigma_{x1} \sigma_{x3} & c \sigma_{x2} \sigma_{x3} & \sigma_{x3} \sigma_{x3}
\end{pmatrix}
$$


(note: this matrix is symmetric and $ \text{cov}(x_i, x_j) =  \text{cov}(x_j, x_i)$, as well as $ \rho(x_i, x_j) =  \rho(x_j, x_i)$)

**Example**

Let's create samle of size 10 with 3 variables $X_1, X_2, X_3$ with means [2, 100, 10] and standard deviations [1, 5.5, 2.3] with the following correlations:

$\rho(X_1, X_2) = 0.3$, $\rho(X_1, X_3) = 0.8$, $\rho(X_2, X_3) = 0.45$

In [ ]:
means = np.array([2, 100, 10])
sds = np.array([1, 5.5, 2.3])

corr_matrix = np.array([
    [1, 0.3, 0.8],
    [0.3, 1, 0.45],
    [0.8, 0.45, 1]])

sds_mult = np.array([
    [sds[0] * sds[0], sds[0] * sds[1], sds[0] * sds[2]],
    [sds[1] * sds[0], sds[1] * sds[1], sds[1] * sds[2]],
    [sds[2] * sds[0], sds[2] * sds[1], sds[2] * sds[2]]])

# more accurate: sds_mult = np.outer(sds, sds)

cov_matrix = corr_matrix * sds_mult
data = np.random.multivariate_normal(means, cov_matrix, size=10)

In [ ]:
#checking correlations

np.corrcoef(data, rowvar=False)

In [ ]:
# sampling more observations
data = np.random.multivariate_normal(means, cov_matrix, size=1000)

#checking correlations
print(np.corrcoef(data, rowvar=False))

**More automatic way to create covariance matrix**

In [ ]:
# function for creating dataset with correlated variables from normal distribution

def create_correlated_variables(num_variables, num_samples, means, sds, corr_matrix):
    assert means.shape[0] == num_variables, f"There should be {num_samples} mean values for num_samples={num_samples}, got {means.shape[0]}"
    assert sds.shape[0] == num_variables, f"There should be {num_samples} sds values for num_samples={num_samples}, got {sds.shape[0]}"
    assert corr_matrix.shape[0] == corr_matrix.shape[1] == num_variables, f"Size of the corr_matrix should be ({num_variables},{num_variables}), got {corr_matrix.shape}"
    assert (corr_matrix <= 1).all() and (corr_matrix >= -1).all(), "All values in corr_matrix should be between -1 and 1"
    
    sds_mult = np.outer(sds, sds)
    cov_matrix = corr_matrix * sds_mult
    
    return np.random.multivariate_normal(means, cov_matrix, size=num_samples)

In [ ]:
# usage example
means = np.array([2, 100, 10])
sds = np.array([1, 5.5, 2.3])

corr_matrix = np.array([
    [1, 0.3, 0.8],
    [0.3, 1, 0.45],
    [0.8, 0.45, 1]])
x = create_correlated_variables(3, 10000, means, sds, corr_matrix)
np.corrcoef(x, rowvar=False)

**Creating dataset with 2 correlated variables with specified treatment effect**

simulating a randomized experiment - imagine we have 2 groups of people - treated and controlled.

In this very simple model, the intervention would add 5 points to some scale.

In [ ]:
# y0 and y1 from the same distribution
np.random.seed(10)
y_if_control = np.random.normal(10, 2, size=10)
treatment_effect = 5
y_if_treated = np.copy(y_if_control) + treatment_effect

data = pd.DataFrame(np.stack([y_if_control, y_if_treated], axis=-1), columns = ['y0', 'y1'])
data

In [ ]:
# or we can create y0 and y1 from different correlated distributions

means = np.array([10, 10])
sds = np.array([2, 2])
corr_matrix = np.array([
    [1, 0.8],
    [0.8, 1]
])

data1 = create_correlated_variables(2, 10, means, sds, corr_matrix)


treatment_effect = 5

# data[:, 0] - controlled, y0
data1[:, 1] += treatment_effect # treated, y1

data1 = pd.DataFrame(data, columns = ['y0', 'y1'])


# or we can just add some noise from normal distribution 

**random treatment assignment: completely randomized experiment**

in a completely randomized experiment, treatments assigned just randomly. Let's simulate this process.

In [ ]:
# generating treatment assignment randomly from bernoulli dostribution
# assigning treatment to each person with probability 0.5 (this probability may differ)
z = np.random.binomial(1, 0.5, size=len(data))

In [ ]:
z

In [ ]:
data['z'] = z

In [ ]:
data

**imitating population groups**

Now, let's imagine that we also have Age column in our data. 

Let's generate new data with this column:

In [ ]:
sample_size = 100


# age column
age = np.random.normal(40, 10, sample_size)

In [ ]:
plt.hist(age)

In [ ]:
# converting age column to int
age = age.astype(int)

we will have 3 groups/blocks of people (based on their age), and there will be treatment effects in different blocks

In [ ]:
np.min(age), np.max(age)

In [ ]:
np.where(age < 45)

In [ ]:
# 3 groups - [19-30, 30-45, 45-60]
# this if my random division into groups, in real research such things are done accorring to some theory/knowledge
age_groups = np.copy(age)
age_groups[age < 30] = 1
age_groups[(age >= 30)*(age < 45)] = 2
age_groups[age >= 45] = 3

In [ ]:
age_groups

In [ ]:
data_age_groups = pd.DataFrame(age_groups, columns=['age_group'])
data_age_groups

Let's also again generate the potential outcome. But now, let's assume that treatment effect depends on the age group. 

In [ ]:
np.random.seed(10)
y_if_control = np.random.normal(10, 2, size=sample_size)
y_if_treated = np.copy(y_if_control)

data_age_groups['y0'] = y_if_control # y if control
data_age_groups['y1'] = y_if_treated # y if treated, but its not final version, we would add TE here

In [ ]:
data_age_groups

Let's assume that TE(treatment effect) for the 1st group is -2, for the 2nd is 5, and for the third is 10. 

In [ ]:
data_age_groups['y1'] = data_age_groups.apply(lambda x: x.y1 - 2 if x.age_group==1 
                                              else x.y1 + 5 if x.age_group==2 
                                              else x.y1 + 10, 
                                              axis=1)
# you can do the same thing but on raw numpy arrays just by indexing  with age_groups array

and, just for now, let's again simulate completely randomized experiment - assign treatments randomly without conditioning on blocks

In [ ]:
# assigning treatment to each person with probability 0.3
z = np.random.binomial(1, 0.3, size=len(data_age_groups))
data_age_groups['z'] = z

In [ ]:
data_age_groups

**example of rerandomization**

In the previous step, we assigned treatment just randomly across all the participants.

Now let's rerandomize our experiment. Let's assign the treatment depending ob the age group with the following probabilities:

+ 1 group, prob=0.2
+ 2 group, prob=0.4
+ 3 group, prob=0.6

In other words, we group our participant into blocks depending on their age, and then assign treatment for each group with different probailities. Probability of being assigned to the treatment is higher for older people.

In [ ]:
def assign_treatment(age_group):
    if age_group == 1:
        return np.random.binomial(1, 0.2)
    if age_group == 2:
        return np.random.binomial(1, 0.4)
    if age_group == 3:
        return np.random.binomial(1, 0.6)


data_age_groups['z_new'] = data_age_groups['age_group'].apply(assign_treatment)

data_age_groups

## Little summary: data simulation

When simulating data for such task we need to think about:
    
1. **Treatment effect**

Simulating y0 and y1 (how are they connected? from one distribution/correlated variables). TE can be equal to all the people, or depend on a group/some covariate.


2. **Treatment assignment (random/randomization in blocks/matched pairs)**

Sumulating treatment assignment - creating the vector $z$. We can just randomly assign treatment (saw an example), or we can do it randomly in some groups of people (different probablities of treatment for different groups, here a treatment assignment 

3. **Covariables/Covariates**

Can be discrete (categorical variables, or binned continuous variable).